<a href="https://colab.research.google.com/github/laiss33/Stock-Price-Prediction-using-Multilingual-Sentiment-Analysis/blob/main/%5B2%5Dpreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# multilingual sentiment merged

In [ ]:
#Aggregate Sentiment Data
mbert = pd.read_excel("/content/drive/MyDrive/shisun_thesis/5_year_sentiment_finetune_mbert_20240527.xlsx")
mbert['mbert_finetune_predicted_sentiment']=mbert['mbert_finetune_predicted_sentiment_id'].replace([0,1,2],['negative','neutral','positive'])
mbert['date'] = mbert['datetime'].dt.date
mbert_agg = mbert.groupby(['date', 'company', 'mbert_finetune_predicted_sentiment']).size().unstack(fill_value=0).reset_index()

In [ ]:
mbert_agg

mbert_finetune_predicted_sentiment,date,company,negative,neutral,positive
0,2018-10-16,YTL Power International,0,0,1
1,2018-10-23,YTL Power International,0,0,1
2,2018-10-27,YTL Power International,0,1,0
3,2018-11-03,YTL Power International,0,1,0
4,2018-11-04,YTL Power International,0,1,0
...,...,...,...,...,...
9860,2023-12-30,PChem (Petronas Chemicals Group),0,1,0
9861,2023-12-30,Tenaga Nasional,0,1,0
9862,2023-12-30,YTL Power International,0,1,0
9863,2023-12-31,CIMB Group,0,2,0


In [ ]:
stock_data = pd.read_excel("/content/drive/MyDrive/shisun_thesis/5years_historical_price.xlsx")

In [ ]:
stock_data

,Unnamed: 0,Date,Close,High,Low,Open,Volume,company_code,company_name
0,0,2019-01-02,4.504456,4.576208,4.480538,4.560263,6022200,1023,IHH Healthcare
1,1,2019-01-02,6.442893,6.518369,6.436031,6.518369,4096900,1155,Maybank
2,2,2019-01-02,4.024759,4.050873,4.014967,4.050873,15246000,1295,YTL Power International
3,3,2019-01-02,5.182747,5.340999,5.127359,5.340999,720000,3816,Celcomdigi
4,4,2019-01-02,131.983246,133.152834,131.533405,133.152834,10600,4707,(HLBank) Hong Leong Bank
...,...,...,...,...,...,...,...,...,...
12245,12245,2023-12-29,7.108116,7.197464,7.108116,7.167681,3103100,5183,PBBank (Public Bank Bhd)
12246,12246,2023-12-29,5.975000,5.994818,5.935365,5.994818,5265700,5225,Tenaga Nasional
12247,12247,2023-12-29,9.795122,9.853659,9.746341,9.746341,3002900,5347,CIMB Group
12248,12248,2023-12-29,18.655687,18.714911,18.438532,18.556980,1446700,5819,PChem (Petronas Chemicals Group)


In [ ]:
# Convert the 'Date' column in 'stock_data' DataFrame to datetime64[ns]
mbert_agg['date'] = pd.to_datetime(mbert_agg['date'])

#Merge Sentiment Data with Stock Price Data
stock_price_with_mbert = pd.merge(stock_data, mbert_agg, how='left', left_on=['Date', 'company_name'], right_on=['date', 'company'])
stock_price_with_mbert.drop(['date', 'company'], axis=1, inplace=True)

stock_price_with_mbert['Total_news'] = stock_price_with_mbert['positive'] + stock_price_with_mbert['neutral'] + stock_price_with_mbert['negative']
stock_price_with_mbert

,Unnamed: 0,Date,Close,High,Low,Open,Volume,company_code,company_name,negative,neutral,positive,Total_news
0,0,2019-01-02,4.504456,4.576208,4.480538,4.560263,6022200,1023,IHH Healthcare,1.0,0.0,1.0,2.0
1,1,2019-01-02,6.442893,6.518369,6.436031,6.518369,4096900,1155,Maybank,2.0,0.0,6.0,8.0
2,2,2019-01-02,4.024759,4.050873,4.014967,4.050873,15246000,1295,YTL Power International,NaN,NaN,NaN,NaN
3,3,2019-01-02,5.182747,5.340999,5.127359,5.340999,720000,3816,Celcomdigi,NaN,NaN,NaN,NaN
4,4,2019-01-02,131.983246,133.152834,131.533405,133.152834,10600,4707,(HLBank) Hong Leong Bank,1.0,0.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12245,12245,2023-12-29,7.108116,7.197464,7.108116,7.167681,3103100,5183,PBBank (Public Bank Bhd),NaN,NaN,NaN,NaN
12246,12246,2023-12-29,5.975000,5.994818,5.935365,5.994818,5265700,5225,Tenaga Nasional,0.0,4.0,6.0,10.0
12247,12247,2023-12-29,9.795122,9.853659,9.746341,9.746341,3002900,5347,CIMB Group,0.0,8.0,5.0,13.0
12248,12248,2023-12-29,18.655687,18.714911,18.438532,18.556980,1446700,5819,PChem (Petronas Chemicals Group),0.0,1.0,4.0,5.0


In [ ]:
stock_price_with_mbert=stock_price_with_mbert.sort_values(by=['company_name','Date'])

In [ ]:
stock_price_with_mbert['Total_news_past_10days']=stock_price_with_mbert.groupby('company_name').rolling('10d', on='Date')['Total_news'].sum().shift(1).to_numpy()
stock_price_with_mbert['Total_Neutral_news_past_10days']=stock_price_with_mbert.groupby('company_name').rolling('10d', on='Date')['neutral'].sum().shift(1).to_numpy()
stock_price_with_mbert['Total_Positve_news_past_10days']=stock_price_with_mbert.groupby('company_name').rolling('10d', on='Date')['positive'].sum().shift(1).to_numpy()
stock_price_with_mbert['Total_Negative_news_past_10days']=stock_price_with_mbert.groupby('company_name').rolling('10d', on='Date')['negative'].sum().shift(1).to_numpy()

In [ ]:
# Step 4: Add New Column by Formula
stock_price_with_mbert['sentiment_ratio'] = (stock_price_with_mbert['Total_Positve_news_past_10days'] - stock_price_with_mbert['Total_Negative_news_past_10days']) / stock_price_with_mbert['Total_news_past_10days']

In [ ]:
stock_price_with_mbert[(stock_price_with_mbert.Total_news_past_10days.notnull()) & (stock_price_with_mbert.company_name=='Maybank')].sort_values(by='Date').head()

,Unnamed: 0,Date,Close,High,Low,Open,Volume,company_code,company_name,negative,neutral,positive,Total_news,Total_news_past_10days,Total_Neutral_news_past_10days,Total_Positve_news_past_10days,Total_Negative_news_past_10days,sentiment_ratio
1,1,2019-01-02,6.442893,6.518369,6.436031,6.518369,4096900,1155,Maybank,2.0,0.0,6.0,8.0,9.0,2.0,6.0,1.0,0.555556
11,11,2019-01-03,6.422307,6.484060,6.394861,6.394861,6983900,1155,Maybank,0.0,0.0,2.0,2.0,8.0,0.0,6.0,2.0,0.500000
21,21,2019-01-04,6.415447,6.436031,6.388001,6.388001,4276600,1155,Maybank,0.0,2.0,0.0,2.0,10.0,0.0,8.0,2.0,0.600000
31,31,2019-01-07,6.490921,6.525228,6.429167,6.429167,7580400,1155,Maybank,0.0,4.0,3.0,7.0,12.0,2.0,8.0,2.0,0.500000
41,41,2019-01-08,6.497783,6.518367,6.477198,6.511505,6692800,1155,Maybank,1.0,1.0,3.0,5.0,19.0,6.0,11.0,2.0,0.473684


In [ ]:
stock_price_with_mbert.drop(['positive', 'negative','neutral','Total_news','Total_Neutral_news_past_10days','Total_Positve_news_past_10days','Total_Negative_news_past_10days'], axis=1, inplace=True)

In [ ]:
import numpy as np
stock_price_with_mbert.describe(include=[np.number])

,Unnamed: 0,Close,High,Low,Open,Volume,company_code,Total_news_past_10days,sentiment_ratio
count,12250.000000,12250.000000,12250.000000,12250.000000,12250.000000,1.225000e+04,12250.000000,9080.000000,9080.000000
mean,6124.500000,20.401505,20.525754,20.261780,20.394601,6.046678e+06,3960.300000,31.545925,0.438856
std,3536.414733,36.549494,36.710256,36.342296,36.526352,8.983253e+06,1922.262014,24.799442,0.263660
min,0.000000,2.181699,2.274758,2.167913,2.198932,0.000000e+00,1023.000000,1.000000,-1.000000
25%,3062.250000,5.362289,5.412771,5.301126,5.356583,9.154500e+05,1295.000000,12.000000,0.310345
50%,6124.500000,6.993163,7.042937,6.930964,6.995450,3.231550e+06,4945.000000,25.000000,0.456331
75%,9186.750000,14.247959,14.386378,14.146569,14.262222,7.774700e+06,5347.000000,47.000000,0.590909
max,12249.000000,137.212982,137.488131,135.168718,136.387515,2.236875e+08,6033.000000,150.000000,1.000000


In [ ]:
stock_price_with_mbert.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12250 entries, 4 to 12242
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              12250 non-null  int64         
 1   Date                    12250 non-null  datetime64[ns]
 2   Close                   12250 non-null  float64       
 3   High                    12250 non-null  float64       
 4   Low                     12250 non-null  float64       
 5   Open                    12250 non-null  float64       
 6   Volume                  12250 non-null  int64         
 7   company_code            12250 non-null  int64         
 8   company_name            12250 non-null  object        
 9   Total_news_past_10days  9080 non-null   float64       
 10  sentiment_ratio         9080 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 1.1+ MB


In [ ]:
stock_price_with_mbert['Total_news_past_10days'] = stock_price_with_mbert['Total_news_past_10days'].fillna(0)
stock_price_with_mbert['sentiment_ratio'] = stock_price_with_mbert['sentiment_ratio'].fillna(0)

In [ ]:
stock_price_with_mbert.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12250 entries, 4 to 12242
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              12250 non-null  int64         
 1   Date                    12250 non-null  datetime64[ns]
 2   Close                   12250 non-null  float64       
 3   High                    12250 non-null  float64       
 4   Low                     12250 non-null  float64       
 5   Open                    12250 non-null  float64       
 6   Volume                  12250 non-null  int64         
 7   company_code            12250 non-null  int64         
 8   company_name            12250 non-null  object        
 9   Total_news_past_10days  12250 non-null  float64       
 10  sentiment_ratio         12250 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 1.1+ MB


In [ ]:
stock_price_with_mbert.to_excel('/content/drive/MyDrive/shisun_thesis/stock_price_with_mbert.xlsx')

# XLM-RoBERTa sentiment merged

In [ ]:
#Aggregate Sentiment Data
xlm = pd.read_excel("/content/drive/MyDrive/shisun_thesis/5_year_sentiment_finetune_mbert_xlm.xlsx")
xlm['xlm_finetune_predicted_sentiment']=xlm['xlm_finetune_predicted_sentiment_id'].replace([0,1,2],['negative','neutral','positive'])
xlm['date'] = xlm['datetime'].dt.date
xlm_agg = xlm.groupby(['date', 'company', 'xlm_finetune_predicted_sentiment']).size().unstack(fill_value=0).reset_index()

In [ ]:
xlm_agg

xlm_finetune_predicted_sentiment,date,company,negative,neutral,positive
0,2018-10-16,YTL Power International,0,1,0
1,2018-10-23,YTL Power International,0,0,1
2,2018-10-27,YTL Power International,0,1,0
3,2018-11-03,YTL Power International,0,1,0
4,2018-11-04,YTL Power International,0,0,1
...,...,...,...,...,...
9860,2023-12-30,PChem (Petronas Chemicals Group),0,1,0
9861,2023-12-30,Tenaga Nasional,0,1,0
9862,2023-12-30,YTL Power International,0,1,0
9863,2023-12-31,CIMB Group,0,2,0


In [ ]:
stock_data = pd.read_excel("/content/drive/MyDrive/shisun_thesis/5years_historical_price.xlsx")

In [ ]:
stock_data

,Unnamed: 0,Date,Close,High,Low,Open,Volume,company_code,company_name
0,0,2019-01-02,4.504456,4.576208,4.480538,4.560263,6022200,1023,IHH Healthcare
1,1,2019-01-02,6.442893,6.518369,6.436031,6.518369,4096900,1155,Maybank
2,2,2019-01-02,4.024759,4.050873,4.014967,4.050873,15246000,1295,YTL Power International
3,3,2019-01-02,5.182747,5.340999,5.127359,5.340999,720000,3816,Celcomdigi
4,4,2019-01-02,131.983246,133.152834,131.533405,133.152834,10600,4707,(HLBank) Hong Leong Bank
...,...,...,...,...,...,...,...,...,...
12245,12245,2023-12-29,7.108116,7.197464,7.108116,7.167681,3103100,5183,PBBank (Public Bank Bhd)
12246,12246,2023-12-29,5.975000,5.994818,5.935365,5.994818,5265700,5225,Tenaga Nasional
12247,12247,2023-12-29,9.795122,9.853659,9.746341,9.746341,3002900,5347,CIMB Group
12248,12248,2023-12-29,18.655687,18.714911,18.438532,18.556980,1446700,5819,PChem (Petronas Chemicals Group)


In [ ]:
# Convert the 'Date' column in 'stock_data' DataFrame to datetime64[ns]
xlm_agg['date'] = pd.to_datetime(xlm_agg['date'])

#Merge Sentiment Data with Stock Price Data
stock_price_with_xlm = pd.merge(stock_data, xlm_agg, how='left', left_on=['Date', 'company_name'], right_on=['date', 'company'])
stock_price_with_xlm.drop(['date', 'company'], axis=1, inplace=True)

stock_price_with_xlm['Total_news'] = stock_price_with_xlm['positive'] + stock_price_with_xlm['neutral'] + stock_price_with_xlm['negative']

In [ ]:
stock_price_with_xlm=stock_price_with_xlm.sort_values(by=['company_name','Date'])

In [ ]:
stock_price_with_xlm['Total_news_past_10days']=stock_price_with_xlm.groupby('company_name').rolling('10d', on='Date')['Total_news'].sum().shift(1).to_numpy()
stock_price_with_xlm['Total_Negative_news_past_10days']=stock_price_with_xlm.groupby('company_name').rolling('10d', on='Date')['negative'].sum().shift(1).to_numpy()
stock_price_with_xlm['Total_Neutral_news_past_10days']=stock_price_with_xlm.groupby('company_name').rolling('10d', on='Date')['neutral'].sum().shift(1).to_numpy()
stock_price_with_xlm['Total_Positve_news_past_10days']=stock_price_with_xlm.groupby('company_name').rolling('10d', on='Date')['positive'].sum().shift(1).to_numpy()
# stock_price_with_xlm['Total_Negative_news_past_10days']=stock_price_with_xlm.groupby('company_name').rolling('10d', on='Date')['Negative'].sum().shift(1).to_numpy()

In [ ]:
# Step 4: Add New Column by Formula
stock_price_with_xlm['sentiment_ratio'] = (stock_price_with_xlm['Total_Positve_news_past_10days']  - stock_price_with_xlm['Total_Negative_news_past_10days'] )/ stock_price_with_xlm['Total_news_past_10days']

In [ ]:
stock_price_with_xlm[(stock_price_with_xlm.Total_news_past_10days.notnull()) & (stock_price_with_xlm.company_name=='Maybank')].sort_values(by='Date').head()

,Unnamed: 0,Date,Close,High,Low,Open,Volume,company_code,company_name,negative,neutral,positive,Total_news,Total_news_past_10days,Total_Negative_news_past_10days,Total_Neutral_news_past_10days,Total_Positve_news_past_10days,sentiment_ratio
1,1,2019-01-02,6.442893,6.518369,6.436031,6.518369,4096900,1155,Maybank,5.0,1.0,2.0,8.0,9.0,2.0,1.0,6.0,0.444444
11,11,2019-01-03,6.422307,6.484060,6.394861,6.394861,6983900,1155,Maybank,0.0,1.0,1.0,2.0,8.0,5.0,1.0,2.0,-0.375000
21,21,2019-01-04,6.415447,6.436031,6.388001,6.388001,4276600,1155,Maybank,0.0,2.0,0.0,2.0,10.0,5.0,2.0,3.0,-0.200000
31,31,2019-01-07,6.490921,6.525228,6.429167,6.429167,7580400,1155,Maybank,0.0,4.0,3.0,7.0,12.0,5.0,4.0,3.0,-0.166667
41,41,2019-01-08,6.497783,6.518367,6.477198,6.511505,6692800,1155,Maybank,1.0,2.0,2.0,5.0,19.0,5.0,8.0,6.0,0.052632


In [ ]:
stock_price_with_xlm.drop(['positive','neutral','negative','Total_news','Total_Negative_news_past_10days','Total_Neutral_news_past_10days','Total_Positve_news_past_10days'], axis=1, inplace=True)

In [ ]:
stock_price_with_xlm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12250 entries, 4 to 12242
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              12250 non-null  int64         
 1   Date                    12250 non-null  datetime64[ns]
 2   Close                   12250 non-null  float64       
 3   High                    12250 non-null  float64       
 4   Low                     12250 non-null  float64       
 5   Open                    12250 non-null  float64       
 6   Volume                  12250 non-null  int64         
 7   company_code            12250 non-null  int64         
 8   company_name            12250 non-null  object        
 9   Total_news_past_10days  9080 non-null   float64       
 10  sentiment_ratio         9080 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 1.1+ MB


In [ ]:
stock_price_with_xlm['Total_news_past_10days'] = stock_price_with_xlm['Total_news_past_10days'].fillna(0)
stock_price_with_xlm['sentiment_ratio'] = stock_price_with_xlm['sentiment_ratio'].fillna(0)

In [ ]:
stock_price_with_xlm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12250 entries, 4 to 12242
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              12250 non-null  int64         
 1   Date                    12250 non-null  datetime64[ns]
 2   Close                   12250 non-null  float64       
 3   High                    12250 non-null  float64       
 4   Low                     12250 non-null  float64       
 5   Open                    12250 non-null  float64       
 6   Volume                  12250 non-null  int64         
 7   company_code            12250 non-null  int64         
 8   company_name            12250 non-null  object        
 9   Total_news_past_10days  12250 non-null  float64       
 10  sentiment_ratio         12250 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 1.1+ MB


In [ ]:
stock_price_with_xlm.to_excel('/content/drive/MyDrive/shisun_thesis/stock_price_with_xlm.xlsx')